# OpenDiffIt

Compares old files with new files. Tracks progress from old data to new data. Checks new documents for accessibility.

## Step 1: 
Enter location of .csv files

In [1]:
location_of_old_file = "example/findfiles-result_january.csv"
location_of_new_file = "example/findfiles-result_february.csv"
location_for_updated_file = "example/findfiles-result_january-february-diff.csv"

Optional: Location of .csv files with extra metadata

In [2]:
location_of_extra_cols_file = "example/findfiles-result_extras.csv"

## Step 2: 
Choose ```Kernel > Restart Kernel and Run All Cells...``` from the menu

### OpenDiffIt: Install prerequisites

In [3]:
!pip install pyqtwebengine
!pip install pyqt5 --user
!pip install urllib3
!pip install idna

In [4]:
!pip install -r ./requirements.txt
import logging
import pandas as pd
import numpy as np
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

  Using cached requests-2.20.0-py2.py3-none-any.whl (60 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cookiecutter 1.7.3 requires requests>=2.23.0, but you have requests 2.20.0 which is incompatible.


Note: Set max rows to display

In [5]:
pd.set_option('display.max_rows', 10)

### OpenDiffIt: Import .csv files

#### - Import old data

Note: If you have no old data to compare you still must have a file with column names that inclue 'url' and 'hash'.

In [6]:
df_old = pd.read_csv(location_of_old_file, dtype=str)
df_old.replace(np.nan, '...', inplace=True)

In [7]:
df_old.drop_duplicates(subset=['url'], inplace=True)
df_old

,count,url,link_text,from_page_url,Unnamed: 4,hash,diff,comply
0,10,http://joelcrawfordsmith.com/openassessit/demo...,Through the Looking Glass,http://joelcrawfordsmith.com/openassessit/demo...,...,64645bd499d83d7d2b97272d30b29bf2c03d1990,SAME,NO
1,8,http://joelcrawfordsmith.com/openassessit/demo...,Alice's Vacilation between Childhood and Adole...,http://joelcrawfordsmith.com/openassessit/demo...,...,2808d21b122d8187a1a502d84a660c66b6b39357,SAME,YES
2,5,http://joelcrawfordsmith.com/openassessit/demo...,Music for Alice in Wonderland play,http://joelcrawfordsmith.com/openassessit/demo...,...,qqqqq5afca7ba5b2ecef4312d9487a1fabf51bc3,SAME,NO
3,4,http://joelcrawfordsmith.com/openassessit/demo...,The Hunting Shark,http://joelcrawfordsmith.com/openassessit/demo...,...,yyyyy5dd26a2ed8cb6c9a086ac9845cc1edd7ff53,SAME,YES
4,2,http://joelcrawfordsmith.com/openassessit/demo...,Phantasmagoria,http://joelcrawfordsmith.com/openassessit/demo...,...,ea191439aeef786ca9fe946a9996b0f44e988387,SAME,YES


#### - Import new current data

In [8]:
df_new = pd.read_csv(location_of_new_file, dtype=str)
df_new.replace(np.nan, '...', inplace=True)

In [9]:
df_new.drop_duplicates(subset=['url'], inplace=True)
df_new

,count,url,link_text,from_page_url
0,9,http://joelcrawfordsmith.com/openassessit/demo...,Alice in Wonderland,http://joelcrawfordsmith.com/openassessit/demo...
1,7,http://joelcrawfordsmith.com/openassessit/demo...,Through the Looking Glass,http://joelcrawfordsmith.com/openassessit/demo...
2,5,http://joelcrawfordsmith.com/openassessit/demo...,Alice's Vacilation between Childhood and Adole...,http://joelcrawfordsmith.com/openassessit/demo...
3,4,http://joelcrawfordsmith.com/openassessit/demo...,Music for Alice in Wonderland play,http://joelcrawfordsmith.com/openassessit/demo...
4,2,http://joelcrawfordsmith.com/openassessit/demo...,The Hunting Shark,http://joelcrawfordsmith.com/openassessit/demo...
5,2,http://joelcrawfordsmith.com/openassessit/demo...,Phantasmagoria,http://joelcrawfordsmith.com/openassessit/demo...


### OpenDiffIt: Create and Carry over and to new dataframe

#### - OpenDiffIt: Create unique 'hash' for each file in .csv

In [10]:
import requests
import hashlib
import logging

def get_remote_sha_sum(url):
    url = url.split('?')[0]
    print('OpenDiffIt: Hashing file ' + url.rsplit('/', 1)[-1].split('?')[0] + ' ...')
    """ Put remote file in memory and create hash """
    MAXSIZE = 26214400 # 25MB
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            response.raise_for_status()

            if len(response.content) < MAXSIZE:
                sha1 = hashlib.sha1()
                response = response.content
                sha1.update(response)
                return sha1.hexdigest()
            else:
                logging.info('Skipping %s because  %s MB is really big.' % (url, str(MAXSIZE/819200)))
        except requests.exceptions.HTTPError as e:
            print('1')
            print("%(error)s:" % dict(error=e))
            return e
    
    else:
        return "Status " + str(response.status_code)
        

In [11]:
# Create hash for each file
    
df_new['hash'] = df_new['url'].apply(get_remote_sha_sum)
# Discard rows with bad urls
df_new = df_new[~df_new['hash'].str.contains("Error", na=True)]
df_new

OpenDiffIt: Hashing file alices-adventures-in-wonderland.pdf ...
OpenDiffIt: Hashing file through-the-looking-glass.pdf ...
OpenDiffIt: Hashing file between-childhood-and-adolescence.pdf ...
OpenDiffIt: Hashing file music-for-alice-in-wonderland.pdf ...
OpenDiffIt: Hashing file hunting-shark.pdf ...
OpenDiffIt: Hashing file phantasmagoria.pdf ...


,count,url,link_text,from_page_url,hash
0,9,http://joelcrawfordsmith.com/openassessit/demo...,Alice in Wonderland,http://joelcrawfordsmith.com/openassessit/demo...,e161ee334885e7261c02bb9e2d0c8b9fc09414ea
1,7,http://joelcrawfordsmith.com/openassessit/demo...,Through the Looking Glass,http://joelcrawfordsmith.com/openassessit/demo...,64645bd499d83d7d2b97272d30b29bf2c03d1990
2,5,http://joelcrawfordsmith.com/openassessit/demo...,Alice's Vacilation between Childhood and Adole...,http://joelcrawfordsmith.com/openassessit/demo...,2808d21b122d8187a1a502d84a660c66b6b39357
3,4,http://joelcrawfordsmith.com/openassessit/demo...,Music for Alice in Wonderland play,http://joelcrawfordsmith.com/openassessit/demo...,11bfd5afca7ba5b2ecef4312d9487a1fabf51bc3
4,2,http://joelcrawfordsmith.com/openassessit/demo...,The Hunting Shark,http://joelcrawfordsmith.com/openassessit/demo...,c6985dd26a2ed8cb6c9a086ac9845cc1edd7ff53
5,2,http://joelcrawfordsmith.com/openassessit/demo...,Phantasmagoria,http://joelcrawfordsmith.com/openassessit/demo...,ea191439aeef786ca9fe946a9996b0f44e988387


#### - Carry over all column headers from the old spreadsheet to the new one

In [12]:
REQ_COLS = ['diff','comply','notes'] # Required cols
EXPIRED_COLS = ['diff', 'count', 'hash'] # Cols with expired data to exclude
existing_cols = list(set().union(df_old.columns, df_new.columns)) # All cols in both spreads
custom_cols = [col for col in existing_cols if col not in EXPIRED_COLS] # Cols that do not exist
all_cols = list(set().union(existing_cols, REQ_COLS))

for col in all_cols:
    if col not in df_new.columns:
        df_new[col] = "..."
                        
df_new.replace(np.nan, '...', inplace=True)
df_new

,count,url,link_text,from_page_url,hash,comply,notes,diff,Unnamed: 4
0,9,http://joelcrawfordsmith.com/openassessit/demo...,Alice in Wonderland,http://joelcrawfordsmith.com/openassessit/demo...,e161ee334885e7261c02bb9e2d0c8b9fc09414ea,...,...,...,...
1,7,http://joelcrawfordsmith.com/openassessit/demo...,Through the Looking Glass,http://joelcrawfordsmith.com/openassessit/demo...,64645bd499d83d7d2b97272d30b29bf2c03d1990,...,...,...,...
2,5,http://joelcrawfordsmith.com/openassessit/demo...,Alice's Vacilation between Childhood and Adole...,http://joelcrawfordsmith.com/openassessit/demo...,2808d21b122d8187a1a502d84a660c66b6b39357,...,...,...,...
3,4,http://joelcrawfordsmith.com/openassessit/demo...,Music for Alice in Wonderland play,http://joelcrawfordsmith.com/openassessit/demo...,11bfd5afca7ba5b2ecef4312d9487a1fabf51bc3,...,...,...,...
4,2,http://joelcrawfordsmith.com/openassessit/demo...,The Hunting Shark,http://joelcrawfordsmith.com/openassessit/demo...,c6985dd26a2ed8cb6c9a086ac9845cc1edd7ff53,...,...,...,...
5,2,http://joelcrawfordsmith.com/openassessit/demo...,Phantasmagoria,http://joelcrawfordsmith.com/openassessit/demo...,ea191439aeef786ca9fe946a9996b0f44e988387,...,...,...,...


#### - Compare old files to new files using 'hash'

In [13]:
def compare_to_old(row):
    ''' Compare old hash to new hash and migrate relevent data '''    
    is_old = df_old[ df_old['url'] == row['url'] ]
        
    if is_old.empty: 
        row['diff'] = 'NEW'
        row['comply'] = 'UNKNOWN'
        row['notes'] = '...'
    elif is_old.iloc[0]['comply'] == 'SKIP':
        row['diff'] = 'SKIP'
        row['comply'] = 'SKIP'
        row['notes'] = 'Not relevent.'
    else:
            
        if (is_old.iloc[0]['hash'] == row['hash']):
            for col in custom_cols:
                row[col] = is_old.iloc[0][col] or '...'
            row['diff'] = 'SAME'
        elif is_old.iloc[0]['hash'] != row['hash']:
            for col in custom_cols:
                row[col] = is_old.iloc[0][col] or '...'
            row['diff'] = 'UPDATED'
            row['comply'] = 'UNKNOWN'
            row['notes'] = '...'
        else:
            row['diff'] = 'IDK'
            row['comply'] = 'IDK'
            row['notes'] = '...'
    return row

df_new.apply(compare_to_old, axis=1)
df_new.replace(np.nan, '...', inplace=True)

In [14]:
# Move specific required columns to the end togther
cols_at_end = REQ_COLS
df_new = df_new[[c for c in df_new if c not in cols_at_end]  + [c for c in cols_at_end if c in df_new]]
df_new.replace(np.nan, '...', inplace=True)
df_new

,count,url,link_text,from_page_url,hash,Unnamed: 4,diff,comply,notes
0,9,http://joelcrawfordsmith.com/openassessit/demo...,Alice in Wonderland,http://joelcrawfordsmith.com/openassessit/demo...,e161ee334885e7261c02bb9e2d0c8b9fc09414ea,...,NEW,UNKNOWN,...
1,7,http://joelcrawfordsmith.com/openassessit/demo...,Through the Looking Glass,http://joelcrawfordsmith.com/openassessit/demo...,64645bd499d83d7d2b97272d30b29bf2c03d1990,...,SAME,NO,...
2,5,http://joelcrawfordsmith.com/openassessit/demo...,Alice's Vacilation between Childhood and Adole...,http://joelcrawfordsmith.com/openassessit/demo...,2808d21b122d8187a1a502d84a660c66b6b39357,...,SAME,YES,...
3,4,http://joelcrawfordsmith.com/openassessit/demo...,Music for Alice in Wonderland play,http://joelcrawfordsmith.com/openassessit/demo...,11bfd5afca7ba5b2ecef4312d9487a1fabf51bc3,...,UPDATED,UNKNOWN,...
4,2,http://joelcrawfordsmith.com/openassessit/demo...,The Hunting Shark,http://joelcrawfordsmith.com/openassessit/demo...,c6985dd26a2ed8cb6c9a086ac9845cc1edd7ff53,...,UPDATED,UNKNOWN,...
5,2,http://joelcrawfordsmith.com/openassessit/demo...,Phantasmagoria,http://joelcrawfordsmith.com/openassessit/demo...,ea191439aeef786ca9fe946a9996b0f44e988387,...,SAME,YES,...


### OpenDiffIt: Check 'NEW' and 'UPDATED' files for compliance

In [15]:
import os
import hashlib
import requests
import wget
import logging
from pdfminer3.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer3.pdfdevice import TagExtractor
from pdfminer3.pdfpage import PDFPage
from io import BytesIO
from urllib.parse import unquote
    
    
def detect_tags(url):
    """ Detect if PDF has proper tags """
    
    tags = ["<b\'Part", "<b\'Sect", "<b\'Art", "<b\'Content", "<b\'Index", "<b\'BibEntry", "<b\'Lbl", "<b\'Index", "<b\'Note", "<b\'Reference", "<b\'Figure", "<b\'Artifact", "<b\'ArtifactSpan", "<b\'LBody", "<b\'Normal", "<b\'Heading 1", "<b\'Heading 2", "<b\'H1", "<b\'H2", "<b\'Table", "<b\'P", "\'Annots"]

    clean_url = unquote(url)
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    
    if clean_url.endswith('.pdf'):
    

        try:
            device = TagExtractor(rsrcmgr, retstr, codec='utf-8')
        except UnicodeError as ex:
            device = TagExtractor(rsrcmgr, retstr, codec='ascii')

        file_name = clean_url.rsplit('/', 1)[-1].split('?')[0]

        temp_download_file_location = os.path.join('tmp', file_name)

        if os.path.exists(temp_download_file_location):
            print("OpenDiffIt: " + file_name + " Using local cached file.")

        else:
            try:
                temp_download_file_location = wget.download(clean_url, temp_download_file_location)
                print("OpenDiffIt: " + file_name + " Streaming new file from server...")
            except Exception as ex:
                print(ex)

        try:
            # Open the file
            with open(temp_download_file_location, 'rb') as fp:
                print('OpenDiffIt: Checking ' + file_name + ' for tags...')

                fp_size = os.path.getsize(temp_download_file_location)

                MAXSIZE = 2306866 # 22MB

                if fp_size < MAXSIZE:
                    logging.info('File is less than 22 MB. Try to detect.')

                    if (fp_size < MAXSIZE / 2):
                        maxpages = 2
                    elif (fp_size < MAXSIZE / 4):
                        maxpages = 4
                    elif (fp_size < MAXSIZE / 8):
                        maxpages = 8
                    else:
                        maxpages = 1

                    interpreter = PDFPageInterpreter(rsrcmgr, device)
                    password = ''
                    caching = True
                    pagenos=set()
                    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching, check_extractable=True):
                        interpreter.process_page(page)

                    contents = retstr.getvalue().decode()
                    device.close() # check if these need to be here still context manager stuff
                    retstr.close() # check if these need to be here still

                    try:
                        if any(item in contents for item in tags):
                            return label_comply(contents)
                        else:
                            return 'NO', 'Needs Tagged.'
                        
                    except Exception as ex:
                        print(ex)

                else:
                    try:
                        print('OpenDiffIt: Too big to scan.')
                        return 'UNKNOWN', 'Too big to scan.'
                    except Exception as ex:
                        print(ex)
                
            try:
                os.remove(temp_download_file_location)
            except Exception as ex:
                print(ex)
                
        except Exception as ex:
            print(ex)
            
    else:
        return 'Not PDF', 'Manually check.'

def label_comply(contents):
    """examine the contents of the file"""
    
    try:
        msg = "Is Tagged. "

        if ("<b'H" in contents):
            msg = msg + " And has a Heading Tag."
            logging.info(msg)                    
            status = 'MAYBE'
            notes = msg

        else:
            msg = msg + " But needs a Heading Tag. Other issues possible."
            logging.info(msg)
            status = 'NO'
            notes = msg

        if ("<b'Table" in contents) and ("<b'TH" not in contents):
            msg = msg + " At least one Table is missing TH."
            logging.info(msg)
            status = 'NO'
            notes = msg

        if "_____" in contents:
            msg = msg + " Probably a Form with issues."
            status = 'NO'
            notes = msg

        if "....." in contents:
            msg = msg + " Probably has a table of contents with issues."
            status = 'NO'
            notes = msg

        return status, msg

    except Exception as ex:
        print(ex)

In [17]:
df_merged = df_new
same = ['SAME', 'SKIP']
other = ['UPDATED', 'NEW', 'IDK']
df_merged_same = df_merged[df_merged.loc[:,'diff'].isin(same)]
df_merged_other = df_merged[df_merged.loc[:,'diff'].isin(other)]


df_merged_other.loc[:,'comply'],df_merged_other.loc[:,'notes'] = zip(*df_merged_other.loc[:,"url"].map(detect_tags))

OpenDiffIt: alices-adventures-in-wonderland.pdf Using local cached file.
OpenDiffIt: Checking alices-adventures-in-wonderland.pdf for tags...
OpenDiffIt: music-for-alice-in-wonderland.pdf Streaming new file from server...
OpenDiffIt: Checking music-for-alice-in-wonderland.pdf for tags...
OpenDiffIt: hunting-shark.pdf Streaming new file from server...
OpenDiffIt: Checking hunting-shark.pdf for tags...


In [18]:
df_merged

,count,url,link_text,from_page_url,hash,Unnamed: 4,diff,comply,notes
0,9,http://joelcrawfordsmith.com/openassessit/demo...,Alice in Wonderland,http://joelcrawfordsmith.com/openassessit/demo...,e161ee334885e7261c02bb9e2d0c8b9fc09414ea,...,NEW,UNKNOWN,...
1,7,http://joelcrawfordsmith.com/openassessit/demo...,Through the Looking Glass,http://joelcrawfordsmith.com/openassessit/demo...,64645bd499d83d7d2b97272d30b29bf2c03d1990,...,SAME,NO,...
2,5,http://joelcrawfordsmith.com/openassessit/demo...,Alice's Vacilation between Childhood and Adole...,http://joelcrawfordsmith.com/openassessit/demo...,2808d21b122d8187a1a502d84a660c66b6b39357,...,SAME,YES,...
3,4,http://joelcrawfordsmith.com/openassessit/demo...,Music for Alice in Wonderland play,http://joelcrawfordsmith.com/openassessit/demo...,11bfd5afca7ba5b2ecef4312d9487a1fabf51bc3,...,UPDATED,UNKNOWN,...
4,2,http://joelcrawfordsmith.com/openassessit/demo...,The Hunting Shark,http://joelcrawfordsmith.com/openassessit/demo...,c6985dd26a2ed8cb6c9a086ac9845cc1edd7ff53,...,UPDATED,UNKNOWN,...
5,2,http://joelcrawfordsmith.com/openassessit/demo...,Phantasmagoria,http://joelcrawfordsmith.com/openassessit/demo...,ea191439aeef786ca9fe946a9996b0f44e988387,...,SAME,YES,...


In [19]:
df_final = pd.concat([df_merged_same, df_merged_other]).sort_index(ascending=True)
df_final

,count,url,link_text,from_page_url,hash,Unnamed: 4,diff,comply,notes
0,9,http://joelcrawfordsmith.com/openassessit/demo...,Alice in Wonderland,http://joelcrawfordsmith.com/openassessit/demo...,e161ee334885e7261c02bb9e2d0c8b9fc09414ea,...,NEW,NO,Needs Tagged.
1,7,http://joelcrawfordsmith.com/openassessit/demo...,Through the Looking Glass,http://joelcrawfordsmith.com/openassessit/demo...,64645bd499d83d7d2b97272d30b29bf2c03d1990,...,SAME,NO,...
2,5,http://joelcrawfordsmith.com/openassessit/demo...,Alice's Vacilation between Childhood and Adole...,http://joelcrawfordsmith.com/openassessit/demo...,2808d21b122d8187a1a502d84a660c66b6b39357,...,SAME,YES,...
3,4,http://joelcrawfordsmith.com/openassessit/demo...,Music for Alice in Wonderland play,http://joelcrawfordsmith.com/openassessit/demo...,11bfd5afca7ba5b2ecef4312d9487a1fabf51bc3,...,UPDATED,NO,Needs Tagged.
4,2,http://joelcrawfordsmith.com/openassessit/demo...,The Hunting Shark,http://joelcrawfordsmith.com/openassessit/demo...,c6985dd26a2ed8cb6c9a086ac9845cc1edd7ff53,...,UPDATED,MAYBE,Is Tagged. And has a Heading Tag.
5,2,http://joelcrawfordsmith.com/openassessit/demo...,Phantasmagoria,http://joelcrawfordsmith.com/openassessit/demo...,ea191439aeef786ca9fe946a9996b0f44e988387,...,SAME,YES,...


#### - _OPTIONAL: Add link and Scrapy location metadata_

In [20]:
df_extra = pd.read_csv(location_of_extra_cols_file, dtype=str)
df_extra

,url,link_text,from_page_url
0,http://joelcrawfordsmith.com/openassessit/demo...,Alice in Wonderland,http://joelcrawfordsmith.com/openassessit/demo...
1,http://joelcrawfordsmith.com/openassessit/demo...,Through the Looking Glass,http://joelcrawfordsmith.com/openassessit/demo...
2,http://joelcrawfordsmith.com/openassessit/demo...,Alice's Vacilation between Childhood and Adole...,http://joelcrawfordsmith.com/openassessit/demo...
3,http://joelcrawfordsmith.com/openassessit/demo...,Music for Alice in Wonderland play,http://joelcrawfordsmith.com/openassessit/demo...
4,http://joelcrawfordsmith.com/openassessit/demo...,The Hunting Shark,http://joelcrawfordsmith.com/openassessit/demo...
5,http://joelcrawfordsmith.com/openassessit/demo...,Phantasmagoria,http://joelcrawfordsmith.com/openassessit/demo...


In [21]:
df_final = df_final.merge(df_extra, how="outer", on="url", suffixes=('','_delme'))
df_final = df_final[[c for c in df_final.columns if not c.endswith('_delme')]]
df_final

,count,url,link_text,from_page_url,hash,Unnamed: 4,diff,comply,notes
0,9,http://joelcrawfordsmith.com/openassessit/demo...,Alice in Wonderland,http://joelcrawfordsmith.com/openassessit/demo...,e161ee334885e7261c02bb9e2d0c8b9fc09414ea,...,NEW,NO,Needs Tagged.
1,7,http://joelcrawfordsmith.com/openassessit/demo...,Through the Looking Glass,http://joelcrawfordsmith.com/openassessit/demo...,64645bd499d83d7d2b97272d30b29bf2c03d1990,...,SAME,NO,...
2,5,http://joelcrawfordsmith.com/openassessit/demo...,Alice's Vacilation between Childhood and Adole...,http://joelcrawfordsmith.com/openassessit/demo...,2808d21b122d8187a1a502d84a660c66b6b39357,...,SAME,YES,...
3,4,http://joelcrawfordsmith.com/openassessit/demo...,Music for Alice in Wonderland play,http://joelcrawfordsmith.com/openassessit/demo...,11bfd5afca7ba5b2ecef4312d9487a1fabf51bc3,...,UPDATED,NO,Needs Tagged.
4,2,http://joelcrawfordsmith.com/openassessit/demo...,The Hunting Shark,http://joelcrawfordsmith.com/openassessit/demo...,c6985dd26a2ed8cb6c9a086ac9845cc1edd7ff53,...,UPDATED,MAYBE,Is Tagged. And has a Heading Tag.
5,2,http://joelcrawfordsmith.com/openassessit/demo...,Phantasmagoria,http://joelcrawfordsmith.com/openassessit/demo...,ea191439aeef786ca9fe946a9996b0f44e988387,...,SAME,YES,...


#### - Replace empty cells with elipse

In [22]:
df_final.replace(np.nan, '...', inplace=True)
df_final

,count,url,link_text,from_page_url,hash,Unnamed: 4,diff,comply,notes
0,9,http://joelcrawfordsmith.com/openassessit/demo...,Alice in Wonderland,http://joelcrawfordsmith.com/openassessit/demo...,e161ee334885e7261c02bb9e2d0c8b9fc09414ea,...,NEW,NO,Needs Tagged.
1,7,http://joelcrawfordsmith.com/openassessit/demo...,Through the Looking Glass,http://joelcrawfordsmith.com/openassessit/demo...,64645bd499d83d7d2b97272d30b29bf2c03d1990,...,SAME,NO,...
2,5,http://joelcrawfordsmith.com/openassessit/demo...,Alice's Vacilation between Childhood and Adole...,http://joelcrawfordsmith.com/openassessit/demo...,2808d21b122d8187a1a502d84a660c66b6b39357,...,SAME,YES,...
3,4,http://joelcrawfordsmith.com/openassessit/demo...,Music for Alice in Wonderland play,http://joelcrawfordsmith.com/openassessit/demo...,11bfd5afca7ba5b2ecef4312d9487a1fabf51bc3,...,UPDATED,NO,Needs Tagged.
4,2,http://joelcrawfordsmith.com/openassessit/demo...,The Hunting Shark,http://joelcrawfordsmith.com/openassessit/demo...,c6985dd26a2ed8cb6c9a086ac9845cc1edd7ff53,...,UPDATED,MAYBE,Is Tagged. And has a Heading Tag.
5,2,http://joelcrawfordsmith.com/openassessit/demo...,Phantasmagoria,http://joelcrawfordsmith.com/openassessit/demo...,ea191439aeef786ca9fe946a9996b0f44e988387,...,SAME,YES,...


### OpenDiffIt: Export dataframe as new file

In [23]:
df_final = df_final[['count','url','hash','link_text','from_page_url','diff','comply','notes']]

In [24]:
df_final.to_csv(location_for_updated_file, index=False)

#### - OpenDiffIt: Export XLSX file with color coded conditional formatting

In [25]:
import csv
import xlsxwriter

def csv_to_xlsx(csv_file):
    """ Convert csv to xlsx with formating """
    
    # Calulate column letter for 'comply' column
    num_of_cols = len(df_final.columns)
    comply_col_index = num_of_cols - 1
    comply_col_letter = chr(ord('@')+comply_col_index)
        
    wb = xlsxwriter.Workbook(csv_file[:-4] + '.xlsx')
    ws = wb.add_worksheet("WS1")    # your worksheet title here
    # ws.insert_textbox('B2', 'Edit using Online Excel in Box!', {'width': 256, 'height': 100})
    ws.insert_textbox('G1', 'Only edit using Online Excel in Box!',
                         {'width': 250,
                          'height': 30,
                          'y_offset': 25,
                          'x_offset': 25,
                          'font': {'bold': True,'color': 'red'},
                          'align': {'vertical': 'middle','horizontal': 'center'},
                          'line': {'color': 'red','width': 1.25,'dash_type': 'square_dot'}})

    formatyellow = wb.add_format({'bg_color':'#FFD960'})
    formatpink = wb.add_format({'bg_color':'#ffc0cb'})
    formatgreen = wb.add_format({'bg_color':'#ccff80'})
    formatgrey = wb.add_format({'bg_color':'#676767'})

    # TODO: Do something with goofy character issues other than ignore errors
    with open(csv_file,'r', encoding='utf-8', errors='ignore') as csvfile:
        """ Convert csv to xlsx with formating """
        table = csv.reader(csvfile)
        i = 0
        # write each row from the csv file as text into the excel file
        # this may be adjusted to use 'excel types' explicitly (see xlsxwriter doc)
        for row in table:
            ws.write_row(i, 0, row)
            i += 1
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="UNKNOWN"',
                      'format':formatyellow})
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="MAYBE"',
                      'format':formatyellow})
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="PENDING"',
                      'format':formatyellow})
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="NO"',
                      'format':formatpink})
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="YES"',
                      'format':formatgreen})
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="SKIP"',
                      'format':formatgreen})

        ws.set_column(0, 0, 50) # url
        ws.set_column(1, 1, 50) # link text
        ws.freeze_panes(1, 0)
    logging.info('Converted csv to pretty xlsx')
    wb.close()

In [27]:
csv_to_xlsx(location_for_updated_file)

## User step 3: Get your resulting CSV and XLSX file

Result file is in the following relative location:

In [28]:
location_for_updated_file

'example/findfiles-result_january-february-diff.csv'